In [ ]:
#this generator uses Generator_fixedValueMultiplier & Generator_Dotproduct_SI_HR_AXIStream & CordicCoreVerilogmodule

In [ ]:
import math
import os
import sys
import shutil
import import_ipynb
from FixedPoint import FXfamily, FXnum
from Generator_DotProduct_N_SI_HR_v4_6_AXI_Stream_function import DotProduct_SI_AXI_Stream 
from Generator_DotProduct_noHR_AXIStream import DotProuct_noHR_AXIStream
from Generator_FixedValueMultiplier import Fixed_Value_Multiplier

In [ ]:
N=input('enter N, number of dotproduct elements (empty for 2): ')
if not N:
    N=2
else:
    N= int(N) 
IN_WIDTH=input('enter width of each elements in bits (empty for 10): ')
if not IN_WIDTH:
     IN_WIDTH = 10
else:
    IN_WIDTH = int(IN_WIDTH)
#hardware reuse value will reduce for sub determinants(dotproducts).
HRR=input('enter hardware reuse rate(2>= , empty or 1 for no HardwareReuse):')
HRFlag=False
if not HRR:
    HRFlag=False
else:   
    HRR = int(HRR)
    if HRR==1:
        HRFlag=False
    if (HRR<2 and HRR>1):
        HRR = 2
        HRFlag=True
    if HRR>=2:
        HRFlag=True 
        if(HRR>=N):
            HRR=N
if(HRFlag==True):#for Hardware Reuse activated conditions
    NMult = math.ceil(N/HRR)
    while math.ceil(N/NMult) != HRR:
        HRR -= 1
    print("HRR calculated value: {}".format(HRR))
    #MAMCS>=1
    MAMCS=input('enter MAMCS for dotproduct:(empty for {})'.format(NMult))  
    if not MAMCS:
        MAMCS = NMult
    else:
        MAMCS = int(MAMCS)
        if MAMCS>NMult:
            MAMCS = NMult
        elif MAMCS<1:
            MAMCS = 1
    Enable_Input_Latch =input("enter ENABLE_INPUT_LATCH(empty or zero for disable):")
    NumOfComponents = math.ceil(NMult/MAMCS)
    #AdderSize>=2
    if MAMCS<NMult:
        AdderSize = input('Enter maximum size allowed for base adders (empty for 2): ')
    if MAMCS==NMult:
        AdderSize = 0
    else:
        if not AdderSize:
            AdderSize = 2
        else:
            AdderSize = int(AdderSize)
            if AdderSize<2:
                AdderSize = 2
            if AdderSize>NumOfComponents:
                AdderSize = NumOfComponents
    if not Enable_Input_Latch:
        Enable_Input_Latch=0
    else:
        Enable_Input_Latch = int(Enable_Input_Latch)
    parameters={ 
        "N":N,
        "HRR":HRR,
        "MAMCS":MAMCS,
        "ADDER_SIZE":AdderSize,
        "IN_WIDTH":IN_WIDTH,
        "ENABLE_INPUT_LATCH":Enable_Input_Latch,
        "Adder_Size":AdderSize
    }
else: #for HardwareReuse diactivated conditions.
    #MAMCS>=2
    MAMCS = input('Enter Multiply-Add Max Component Size (>=2, <= {}, empty for 2): '.format(N-1)) 
    #AdderSize>=2
    AdderSize = input('Enter maximum size allowed for base adders (empty for 2): ')
    #INPUT_REG_DEPTH>=0
    if (not MAMCS):
        MAMCS = N-1
    else:
        MAMCS = int(MAMCS)
    if MAMCS>=(N-1):
        MAMCS = N-1
        NumOfMaxSizeComponents=1
        NonMaxSizeComponentSize=1;
    else:
        if(MAMCS<2): 
            MAMCS=2
        NumOfMaxSizeComponents = math.floor(N/MAMCS)
        if (N/MAMCS) == NumOfMaxSizeComponents:
            NonMaxSizeComponentSize=0
        else:
            NonMaxSizeComponentSize = N - MAMCS*NumOfMaxSizeComponents
    NumOfComponents = NumOfMaxSizeComponents + (NonMaxSizeComponentSize!=0)

    if not AdderSize: 
        AdderSize=2
    else:
        AdderSize = int(AdderSize)
        if AdderSize<2:
            AdderSize=2
        if AdderSize>NumOfComponents:
            AdderSize=NumOfComponents
    parameters={ 
        "N":N,
        "MAMCS":MAMCS,
        "ADDER_SIZE":AdderSize,
        "IN_WIDTH":IN_WIDTH,
    }
INPUT_REG_DEPTH = input('Enter INPUT_REG_DEPTH Default Value (empty for 1): ')
#MULT_PIPE_DEPTH>=0
MULT_PIPE_DEPTH = input('Enter MULT_PIPE_DEPTH Default Value (empty for 1): ')
if not INPUT_REG_DEPTH:
    INPUT_REG_DEPTH=1
else:
    INPUT_REG_DEPTH = int(INPUT_REG_DEPTH)
    if(INPUT_REG_DEPTH<0):
        INPUT_REG_DEPTH=0

if not MULT_PIPE_DEPTH:
    MULT_PIPE_DEPTH=1
else:
    MULT_PIPE_DEPTH = int(MULT_PIPE_DEPTH)
    if(MULT_PIPE_DEPTH<0):
        MULT_PIPE_DEPTH=0
parameters["INPUT_REG_DEPTH"]=INPUT_REG_DEPTH
parameters["MULT_PIPE_DEPTH"]=MULT_PIPE_DEPTH
compensationFlag=input("cordic result compensation(1 or empty for true and zero for false)?")
if not compensationFlag:
    compensationFlag=True
else:
    compensationFlag=int(compensationFlag)
    if compensationFlag>=1:
        compensationFlag=True
    else:
        compensationFlag=False
integerPartLenght=input("enter integer part lenght of signed Qm.n fixed point standard:(empty for {})".format(math.ceil(IN_WIDTH/2)))
if not integerPartLenght:
    integerPartLenght=3
else:
    integerPartLenght=int(integerPartLenght)
    if(integerPartLenght<0):
        integerPartLenght=0
    if(integerPartLenght>IN_WIDTH): 
        integerPartLenght=IN_WIDTH
fractionalPartLen=input("enter fractional part lenght of signed Qm.n fixed point standard:(empty for {})".format(abs(IN_WIDTH-integerPartLenght)))
if not fractionalPartLen:
    fractionalPartLen=abs(IN_WIDTH-integerPartLenght)
else:
    fractionalPartLen=int(fractionalPartLen)
    if (fractionalPartLen<0):
        fractionalPartLen=0
    if(fractionalPartLen>IN_WIDTH):
        fractionalPartLen=IN_WIDTH
if(integerPartLenght+fractionalPartLen==IN_WIDTH):
    print("integer Part Lenght is:{} and fractional part lenght is:{}".format(integerPartLenght,fractionalPartLen))
else:
    print("integer and fractional part lenght does not match with input width. re run the code.")
#in the following configuaraion of cordic we can not extend the output width in each iteration and next core INwidth is as previous core.
#port width calculation:
#******************dotproduct port lenght calculation.
lgN=math.ceil(math.log2(N))
IN_DATA_LENGHT_DP=2*N*IN_WIDTH 
OUT_WIDTH_DP=2*IN_WIDTH+lgN
OUT_DATA_LENGHT_DP=OUT_WIDTH_DP
#******************the FVM1 is for subtraction of min value of previous range scaling it to ROC and then addition of ROC min value.
IN_WIDTH_FVM1=OUT_DATA_LENGHT_DP
IN_DATA_LENGHT_FVM1=IN_WIDTH_FVM1
FVM1_INTEGER_PART_LENGHT=2*integerPartLenght+lgN 
FVM1_FRACTIONAL_PART_LENGHT=2*fractionalPartLen
FVM1_WORDLENGHT=FVM1_FRACTIONAL_PART_LENGHT+FVM1_INTEGER_PART_LENGHT
#for simplification in scaling we will narrow the [0.5,2) into [0.75,1.75) which has lenght of 1=2^0 so that the scaling will be a simple shift and bit extend.
#now, the original binary point will convert to Desiered place for ROC just with shift. 
differenceToROC=(FVM1_WORDLENGHT-2)-FVM1_FRACTIONAL_PART_LENGHT#difference of binary point from REGION OF CONVERGENCE(which is two bit for integer and the rest for fractions)
OUT_WIDTH_FVM1=FVM1_WORDLENGHT+abs(differenceToROC)#+2# last two more bit is for bias addition, one in input and one in output.
OUT_DATA_LENGHT_FVM1=OUT_WIDTH_FVM1
#****************** cordic core
numIteration = OUT_WIDTH_FVM1   #for n bit resolution in output we need to do n iterations of cordic on input
MAX_ITERATION_WIDTH=math.ceil(math.log2(numIteration))
IN_WIDTH_Cordic=OUT_DATA_LENGHT_FVM1   #  one more bit is not needed for addition and subtraction of 0.25 for X and Y becuse of no overflow.
OUT_WIDTH_Cordic=IN_WIDTH_Cordic
addSubCordicBiasposition=OUT_DATA_LENGHT_FVM1-4  #position of 0.25 for X=data+0.25 and Y=data-0.25 in the new binary point placement(-4 bring to 4th bit form MSB which is 0.25 bit.).
IN_DATA_LENGHT_Cordic=2*IN_WIDTH_Cordic 
OUT_DATA_LENGHT_Cordic=IN_DATA_LENGHT_Cordic#iteration value for next core will be inserted throw a parameter in instanciatition of the core.
#*****************the FVM2 is for cordic compensation and min value of 0.75 subtraction.
IN_WIDTH_FVM2=OUT_WIDTH_Cordic
OUT_WIDTH_FVM2=IN_WIDTH_FVM2
OUT_DATA_LENGHT_FVM2=OUT_WIDTH_FVM2
FVM2_INTEGER_PART_LENGHT=2#due to what?
FVM2_FRACTIONAL_PART_LENGHT=IN_WIDTH_FVM2-FVM2_INTEGER_PART_LENGHT

#*****************the FVM3 is for Unscaling to previous scail factor. and min value addition.
IN_WIDTH_FVM3=OUT_WIDTH_FVM2
OUT_WIDTH_FVM3=IN_WIDTH_FVM3 
FVM3_INTEGER_PART_LENGHT=FVM2_INTEGER_PART_LENGHT
FVM3_FRACTIONAL_PART_LENGHT=FVM2_FRACTIONAL_PART_LENGHT
#if(differenceToROC>=0):

    #scale to lower range
#else:
    #scale to higher range   
#*******************Cordic gain calculation:
cordicGain=1
for i in range (numIteration):
    cordicGain*=math.sqrt(1-pow(2,-2*(i+1)))
    #print('cordic gain after {} iteration: {}'.format(i+1,cordicGain))
print('\n')
compensationScale=1/cordicGain
print('compensation scale after {} iteration: {}'.format(numIteration,compensationScale))
#*******************FVMs scale and biases calculation.
scaleFVM1=1#pow(2,-fractionalPartLen)/pow(2,-(FVM1_WORDLENGHT-2))
print("scale fvm1:{}".format(scaleFVM1))
if(compensationFlag==True):
    scaleFVM2=compensationScale
else:
    scaleFVM2=1
print("scale fvm2:{}".format(scaleFVM2))
scaleFVM3=1#1/2*scaleFVM1
print("scale fvm3:{}".format(scaleFVM3))
#update: no bias should be in data. if not the square root is not valid due to scailing formula.
inbiasFVM1=0#pow(2,OUT_WIDTH_DP) #it should be subtracted from smallest number possible in the  previos range(which is negetive).
outbiasFVM1=0#0.75#it should be added to smallest number possible in the desired range(0.75).
inbiasFVM2=0#0 #no bias needed.
outbiasFVM2=0#0 #no bias needed.
inbiasFVM3=0#-0.75 #it should be subtracted from smallest number possible in the  previos range(which is 0.75).
outbiasFVM3=0#-pow(2,OUT_WIDTH_DP) #it should be added to smallest number possible in the desired range(which is negetive).
#*******************File directory operation
Files_Location="secondNorm_GeneratedVerilogCodes/"
Print_To_File=1
Is_Top=1
modules={}

ModuleName='vectorSecondNorm'
ModuleName+='_N_{}'.format(N)
ModuleName+='_{}bits'.format(IN_WIDTH)
if(HRFlag):
    ModuleName+='_HRdpx{}'.format(HRR)
else:
    ModuleName+='_noHR'
if(compensationFlag):
    ModuleName+='_Com'
else:
    ModuleName+='_NCom'

ModuleName+='_Q{}_{}_AXIStream'.format(integerPartLenght,fractionalPartLen)
if Is_Top:
    Files_Location += ModuleName+"/"
    if not os.path.exists(Files_Location):
        os.makedirs(Files_Location)
#CordicModuleSourceLocation="C:/Users/Delavar/OneDrive/thesis/abbasi/Scripts_/Scripts_/Generator/CordicCoreAXIStream.v"
#CordicModuleDestinationLocation=Files_Location
#shutil.copyfile(CordicModuleLocation,Files_Location)
CoridcModuleName="CORDICCore"
if Print_To_File<=0:
    of = sys.stdout
else:
    of = open(Files_Location+ModuleName+".v", 'w+')
fileObject=of
#sub modules creation
ModuleNameFVM1=Fixed_Value_Multiplier(inbiasFVM1,outbiasFVM1,scaleFVM1,FVM1_INTEGER_PART_LENGHT,FVM1_FRACTIONAL_PART_LENGHT,IN_WIDTH_FVM1,OUT_WIDTH_FVM1,Files_Location,Print_To_File)
ModuleNameFVM2=Fixed_Value_Multiplier(inbiasFVM2,outbiasFVM2,scaleFVM2,FVM2_INTEGER_PART_LENGHT,FVM2_FRACTIONAL_PART_LENGHT,IN_WIDTH_FVM2,OUT_WIDTH_FVM2,Files_Location,Print_To_File)
ModuleNameFVM3=Fixed_Value_Multiplier(inbiasFVM3,outbiasFVM3,scaleFVM3,FVM3_INTEGER_PART_LENGHT,FVM3_FRACTIONAL_PART_LENGHT,IN_WIDTH_FVM3,OUT_WIDTH_FVM3,Files_Location,Print_To_File)
if(HRFlag):
    dotProductModuleName=DotProduct_SI_AXI_Stream(parameters,Print_To_File,Files_Location,0,modules)
else:
    dotProductModuleName=DotProuct_noHR_AXIStream(parameters,Print_To_File,Files_Location,0,modules)
    print("no HR mode.")

#main module definition.
print("`timescale {}ns / {}ps".format(1,1),file=fileObject)
print("//the compensation scale for this module is: {}".format(compensationScale),file=fileObject)
#print("//latency(clocks) of this module is: {}".format(numIteration),file=of)
print("module "+ModuleName,file=fileObject)
print("#(",file=fileObject)
print("parameter IN_DATA_LENGHT= {}, ".format(IN_DATA_LENGHT_DP),file=fileObject)
print("parameter OUT_DATA_LENGHT= {} ".format(OUT_DATA_LENGHT_FVM2),file=fileObject)
print(")( \n",file=fileObject)
print("input aclk,".format(),file=fileObject)
print("input aresetn,".format(),file=fileObject)
print("input enable,".format(),file=fileObject)
print("input [IN_DATA_LENGHT-1:0]s_axi_data,".format(),file=fileObject)
print("input s_axi_valid,".format(),file=fileObject) 
print("input m_axi_ready,".format(),file=fileObject)
print("output [OUT_DATA_LENGHT-1:0]m_axi_data,".format(),file=fileObject)
print("output m_axi_valid,".format(),file=fileObject)
print("output s_axi_ready".format(),file=fileObject)
print(" );\n".format(),file=fileObject)
#input ports wiring.
print("wire [{}-1:0]data_InputToDotproduct;".format(IN_DATA_LENGHT_DP),file=fileObject)
print("wire ready_InputToDotproduct;",file=fileObject)
print("wire valid_InputToDotproduct;",file=fileObject)
print("assign valid_InputToDotproduct=s_axi_valid;",file=fileObject)
print("assign s_axi_ready=ready_InputToDotproduct;",file=fileObject)
print("assign data_InputToDotproduct=s_axi_data;",file=fileObject)
#output ports wiring.
print("wire [{}-1:0]data_FVM2ToOutput;".format(OUT_DATA_LENGHT_FVM2),file=fileObject)
print("wire ready_FVM2ToOutput;",file=fileObject)
print("wire valid_FVM2ToOutput;",file=fileObject)
print("assign m_axi_valid=valid_FVM2ToOutput;",file=fileObject)
print("assign ready_FVM2ToOutput=m_axi_ready;",file=fileObject)
print("assign m_axi_data=data_FVM2ToOutput;",file=fileObject)
#interconnct wiring and submodules instanciations.
#instanciating sum of square module.
print("wire [{}-1:0]data_DotproductToFVM1;".format(OUT_DATA_LENGHT_DP),file=fileObject)
print("wire ready_DotproductToFVM1;",file=fileObject)
print("wire valid_DotproductToFVM1;",file=fileObject)
print("{} sumofSquares(".format(dotProductModuleName),file=fileObject)
print(".aclk(aclk),",file=fileObject)
print(".aresetn(aresetn),",file=fileObject)
print(".enable(enable),",file=fileObject)
print(".s_axi_data(data_InputToDotproduct),",file=fileObject)
print(".s_axi_ready(ready_InputToDotproduct),",file=fileObject)
print(".s_axi_valid(valid_InputToDotproduct),",file=fileObject)
print(".m_axi_data(data_DotproductToFVM1),",file=fileObject)
print(".m_axi_ready(ready_DotproductToFVM1),",file=fileObject)
print(".m_axi_valid(valid_DotproductToFVM1)",file=fileObject)
print(");",file=fileObject)
#instanciating FVM1 module.
print("wire signed[{}-1:0]data_FVM1ToCordic;".format(OUT_DATA_LENGHT_FVM1),file=fileObject)
print("wire ready_FVM1ToCordic;",file=fileObject)
print("wire valid_FVM1ToCordic;",file=fileObject)
print("{} fixedValueMultiplir1(".format(ModuleNameFVM1),file=fileObject)
print(".aclk(aclk),",file=fileObject)
print(".aresetn(aresetn),",file=fileObject)
print(".enable(enable),",file=fileObject)
print(".s_axi_data(data_DotproductToFVM1) ,",file=fileObject)
print(".s_axi_ready(ready_DotproductToFVM1),",file=fileObject)
print(".s_axi_valid(valid_DotproductToFVM1),",file=fileObject)
print(".m_axi_data(data_FVM1ToCordic),",file=fileObject)
print(".m_axi_ready(ready_FVM1ToCordic),",file=fileObject)
print(".m_axi_valid(valid_FVM1ToCordic)",file=fileObject)
print(");",file=fileObject)
#biasing FVMoutput for cordic x and y arguments.
print("wire signed[{}-1:0]data_FVMToCordicX;".format(OUT_WIDTH_FVM1),file=fileObject)#this wire carry biased data(+0.25) for cordic x input
print("wire signed[{}-1:0]data_FVMToCordicY;".format(OUT_WIDTH_FVM1),file=fileObject)#this wire carry biased data(-0.25) for cordic y input
print("wire signed[{}-1:0] CordicInputbias={{4'b0001,{{{}-4{{1'b0}}}}}};".format(OUT_WIDTH_FVM1,OUT_WIDTH_FVM1),file=fileObject)
print("assign data_FVMToCordicX=data_FVM1ToCordic+CordicInputbias;",file=fileObject)
print("assign data_FVMToCordicY=data_FVM1ToCordic-CordicInputbias;",file=fileObject) 
#instanciating FVM2 module.
print("wire [{}-1:0]data_CordicToFVM2;".format(OUT_DATA_LENGHT_FVM1),file=fileObject)
print("wire ready_CordicToFVM2;",file=fileObject)
print("wire valid_CordicToFVM2;",file=fileObject)
print("{} fixedValueMultiplir2(".format(ModuleNameFVM2),file=fileObject)
print(".aclk(aclk),",file=fileObject)
print(".aresetn(aresetn),",file=fileObject)
print(".enable(enable),",file=fileObject)
print(".s_axi_data(data_CordicToFVM2),",file=fileObject)
print(".s_axi_ready(ready_CordicToFVM2),",file=fileObject)
print(".s_axi_valid(valid_CordicToFVM2),",file=fileObject)
print(".m_axi_data(data_FVM2ToOutput),",file=fileObject)
print(".m_axi_ready(ready_FVM2ToOutput),",file=fileObject)
print(".m_axi_valid(valid_FVM2ToOutput)",file=fileObject)
print(");",file=fileObject)
#instanciating Cordic mashine modules.
ITERATION_VALUE=1
IterationDone=False
REPEAT_ITERATION=4 #4 is the first iteration that should be repeted next is 3k+1.(k0=4)
for i in range(numIteration):
    if(numIteration<2):
        s_axi_data_wire="data_scaleToCordic"
        s_axi_valid_wire="valid_scaleToCordic"
        s_axi_ready_wire="ready_scaleToCordic"
        m_axi_data_wire="data_coridcToUnscale"
        m_axi_valid_wire="valid_coridcToUnscale"
        m_axi_ready_wire="ready_coridcToUnscale"
    elif(i==0):
        s_axi_data_wire="data_scaleToCordic"
        s_axi_valid_wire="valid_scaleToCordic"
        s_axi_ready_wire="ready_scaleToCordic"
        m_axi_data_wire="data_cordic{}to{}".format(i,i+1)
        m_axi_valid_wire="valid_cordic{}to{}".format(i,i+1)
        m_axi_ready_wire="ready_cordic{}to{}".format(i,i+1)
    elif(i==numIteration-1):
        s_axi_data_wire="data_cordic{}to{}".format(i-1,i)
        s_axi_valid_wire="valid_cordic{}to{}".format(i-1,i)
        s_axi_ready_wire="ready_cordic{}to{}".format(i-1,i)
        m_axi_data_wire="data_coridcToUnscale"
        m_axi_valid_wire="valid_coridcToUnscale"
        m_axi_ready_wire="ready_coridcToUnscale"
    else:
        s_axi_data_wire="data_cordic{}to{}".format(i-1,i)
        s_axi_valid_wire="valid_cordic{}to{}".format(i-1,i)
        s_axi_ready_wire="ready_cordic{}to{}".format(i-1,i)
        m_axi_data_wire="data_cordic{}to{}".format(i,i+1)
        m_axi_valid_wire="valid_cordic{}to{}".format(i,i+1)
        m_axi_ready_wire="ready_cordic{}to{}".format(i,i+1)
    if(i==0):    
        print("wire [{}:{}]{};".format(IN_DATA_LENGHT_Cordic-1,0,s_axi_data_wire),file=fileObject)
        print("wire {};".format(s_axi_ready_wire),file=fileObject)
        print("wire {};".format(s_axi_valid_wire),file=fileObject)
        print("wire [{}:{}]{};".format(OUT_DATA_LENGHT_Cordic-1,0,m_axi_data_wire),file=fileObject)
        print("wire {};".format(m_axi_ready_wire),file=fileObject)
        print("wire {};".format(m_axi_valid_wire),file=fileObject)   
    else:
        print("wire [{}:{}]{};".format(OUT_DATA_LENGHT_Cordic-1,0,m_axi_data_wire),file=fileObject)
        print("wire {};".format(m_axi_ready_wire),file=fileObject)
        print("wire {};".format(m_axi_valid_wire),file=fileObject)     
    print("{} #(.MAX_ITERATION_WIDTH({}),.IN_WIDTH({})) cordicCore{}(".format(CoridcModuleName,MAX_ITERATION_WIDTH,IN_WIDTH_Cordic,i),file=fileObject)#import and use cordic module in here.
    print(".aclk(aclk),",file=fileObject)
    print(".aresetn(aresetn),",file=fileObject)
    print(".enable(enable),",file=fileObject)
    print(".s_axi_data({{{},{}'d{}}}),".format(s_axi_data_wire,MAX_ITERATION_WIDTH,ITERATION_VALUE),file=fileObject)
    #print(".s_axi_data({}),".format(s_axi_data_wire),file=fileObject)
    print(".s_axi_ready({}),".format(s_axi_ready_wire),file=fileObject)
    print(".s_axi_valid({}),".format(s_axi_valid_wire),file=fileObject)
    print(".m_axi_data({}),".format(m_axi_data_wire),file=fileObject)
    print(".m_axi_ready({}),".format(m_axi_ready_wire),file=fileObject)
    print(".m_axi_valid({})".format(m_axi_valid_wire),file=fileObject)
    print(");",file=fileObject)
    #first repeating iteration is 4th one, so we should have at least 5 iteration number for having repeats.
    if(ITERATION_VALUE==REPEAT_ITERATION):
        if(IterationDone==True):
            REPEAT_ITERATION=3*REPEAT_ITERATION+1#updating next repeating iteration .
            IterationDone=False #next repeating iteration is not done yet.
            ITERATION_VALUE+=1# previous iteration is repeated and we should continue.
        else:
            IterationDone=True
            ITERATION_VALUE=ITERATION_VALUE
    else:
        ITERATION_VALUE+=1
print("assign data_scaleToCordic={{data_FVMToCordicX,data_FVMToCordicY}};",file=fileObject)
print("assign valid_scaleToCordic=valid_FVM1ToCordic;",file=fileObject)
print("assign ready_scaleToCordic=ready_FVM1ToCordic;",file=fileObject)
print("assign data_CordicToFVM2=data_coridcToUnscale[{}:{}];".format(2*IN_WIDTH_Cordic-1,IN_WIDTH_Cordic),file=fileObject)
print("assign valid_CordicToFVM2=valid_coridcToUnscale;",file=fileObject)
print("assign ready_coridcToUnscale=ready_CordicToFVM2;",file=fileObject)
print("endmodule".format(),file=fileObject)    
if(Print_To_File>=1):
    of.close()

In [ ]:

####TESTING AREA.
wordLen1=8
wordLen2=10
Range1=(pow(2,wordLen1-1)-1)-(-pow(2,wordLen1-1)) +1
Range2=(pow(2,wordLen2-1)-1)-(-pow(2,wordLen2-1)) +1
print(Range1)
print(Range2)
scale=Range2/Range1
unscale=Range1/Range2
print(scale)
print(unscale)
number =1
scalednumber=(number-(-pow(2,wordLen1-1)))*scale+(-pow(2,wordLen2-1))
unscalednumber=(scalednumber-(-pow(2,wordLen2-1)))*unscale+(-pow(2,wordLen1-1))
print(scalednumber)
print(unscalednumber)

In [ ]:
CordicModuleSourceLocation="C:/Users/Delavar/OneDrive/thesis/abbasi/Scripts_/Scripts_/Generator/CordicCoreAXIStream.v"
CordicModuleDestinationLocation="C:/Users/Delavar/OneDrive/thesis/abbasi/Scripts_/Scripts_/Generator/secondNorm_GeneratedVerilogCodes"
shutil.copyfile(CordicModuleLocation,Files_Location)

In [ ]:
OUT_WIDTH_FVM1=10
print("wire signed[{}-1:0] CordicInputbias={{4'b0001,{{{}-4{{0}}}}}}".format(OUT_WIDTH_FVM1,OUT_WIDTH_FVM1))